In [1]:
import warnings
from loguru import logger

warnings.simplefilter(action='ignore', category=FutureWarning)

from edgedroid.models.timings import *
import edgedroid.data as default_data

logger.enable("edgedroid")

data = default_data.load_curve_fitting_data()
data

,participant,prev_ttf,exec_time,prev_duration,neuro,neuroticism
0,134146,0.0,3.029,"[-inf, 5.0)","[-inf, 0.5)",0.375
1,134146,0.0,4.004,"[-inf, 5.0)","[-inf, 0.5)",0.375
2,134146,0.0,2.625,"[-inf, 5.0)","[-inf, 0.5)",0.375
3,134146,0.0,4.680,"[-inf, 5.0)","[-inf, 0.5)",0.375
4,134146,0.0,5.197,"[-inf, 5.0)","[-inf, 0.5)",0.375
...,...,...,...,...,...,...
6755,137353,0.0,6.358,"[5.0, 9.0)","[0.5, inf)",0.625
6756,137353,0.0,3.840,"[5.0, 9.0)","[0.5, inf)",0.625
6757,137353,0.0,2.932,"[9.0, inf)","[0.5, inf)",0.625
6758,137353,0.0,1.874,"[9.0, inf)","[0.5, inf)",0.625


In [2]:
import numpy as np

data["outlier"] = False
for _, df in data.groupby(["prev_ttf", "prev_duration"]):
    outliers = ((df["exec_time"] > np.percentile(df["exec_time"], 95)) | (df["exec_time"] < np.percentile(df["exec_time"], 5)))
    outlier_idx = df.loc[outliers].index
    data.loc[outlier_idx, "outlier"] = True
data

,participant,prev_ttf,exec_time,prev_duration,neuro,neuroticism,outlier
0,134146,0.0,3.029,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
1,134146,0.0,4.004,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
2,134146,0.0,2.625,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
3,134146,0.0,4.680,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
4,134146,0.0,5.197,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
...,...,...,...,...,...,...,...
6755,137353,0.0,6.358,"[5.0, 9.0)","[0.5, inf)",0.625,False
6756,137353,0.0,3.840,"[5.0, 9.0)","[0.5, inf)",0.625,False
6757,137353,0.0,2.932,"[9.0, inf)","[0.5, inf)",0.625,False
6758,137353,0.0,1.874,"[9.0, inf)","[0.5, inf)",0.625,False


In [3]:
from collections import deque
import pandas as pd
from typing import Tuple, Deque
from tqdm.notebook import tqdm
import numpy.typing as npt

def outliers_to_nan_then_mean(a: npt.NDArray | npt.ArrayLike, p: float = 5) -> float:
    upper_bound = np.percentile(a, 100 - p)
    lower_bound = np.percentile(a, p)
    a[a > upper_bound] = np.nan
    a[a < lower_bound] = np.nan
    return np.mean(a)
    
    

result_rows = deque()
for participant in tqdm(data["participant"].unique(), desc="Run"):
    class MultiCurveFittingTestModel(MultiCurveFittingExecutionTimeModel):
        @staticmethod
        def get_data() -> pd.DataFrame:
            data = MultiCurveFittingExecutionTimeModel.get_data()
            return data[data["participant"] != participant].copy()
        
        def __init__(self, neuroticism: float):
            super().__init__(
                neuroticism=neuroticism,
                agg_fn=outliers_to_nan_then_mean
            )
            
    class Legacy(LegacyModel):
        @staticmethod
        def get_data() -> pd.DataFrame:
            data = MultiCurveFittingExecutionTimeModel.get_data()
            return data[data["participant"] != participant].copy()
        
        
    class PowerFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (PowerFit,)
        
    class SquareFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (SquareFit,)
        
    class CubeFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (CubeFit,)
        
    class ExponentialFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (ExponentialFit,)
        
    def get_test_data(*args, **kwargs) -> (
        Tuple[
            pd.DataFrame,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
        ]
    ):
        data, *rest = ExecutionTimeModel.get_data()
        return (data[data["run_id"] != participant].copy(), *rest)

    class NaiveTestModel(FirstOrderETM):
        get_data = get_test_data
        
    class NaiveAggTestModel(FirstOrderAggregateETM):
        get_data = get_test_data
            
    run_data = data[data["participant"] == participant].copy()

    neuroticism = run_data["neuroticism"].unique()[0]
    naive_model = NaiveTestModel()
    naive_mean_model = NaiveAggTestModel(np.mean)
    naive_median_model = NaiveAggTestModel(np.median)
    
    power_fit_model = PowerFitTestModel(neuroticism)
    square_fit_model = SquareFitTestModel(neuroticism)
    cube_fit_model = CubeFitTestModel(neuroticism)
    exponential_fit_model = ExponentialFitTestModel(neuroticism)
    multi_fit_model = MultiCurveFittingExecutionTimeModel(neuroticism)
    
    legacy_model = Legacy()
    
    # deque containing (model name, model object, ttf_bins, window_size, kernel name)
    models: Deque[Tuple[str, ExecutionTimeModel, int, int, str]] = deque()
    models.extend((
        ("1st order", naive_model, 0, 0, "none"),
        ("1st order (median)", naive_median_model, 0, 0, "none"),
        ("1st order (mean)", naive_mean_model, 0, 0, "none"),
        ("a * x^b + c", power_fit_model , 0, 0, "none"),
        ("a * x^2 + b * x + c", square_fit_model, 0, 0, "none"),
        ("a * x^3 + b * x^2 + c * x + d", cube_fit_model, 0, 0, "none"),
        ("a * e^x + b", exponential_fit_model, 0, 0, "none"),
        ("multi curve", multi_fit_model, 0, 0, "none"),
        ("legacy", legacy_model, 0, 0, "none")
    ))
        
    for _ in range(1):
        for i, (prev_ttf, exec_time, neuro, prev_duration, outlier) in enumerate(run_data[["prev_ttf", "exec_time", "neuro", "prev_duration", "outlier"]].itertuples(index=False)):
            for model_name, model, ttf_bins, window_size, kernel_name in models:
                if i == 0:
                    # reset the model between runs
                    model.reset()
                else:
                    # first row has no previous ttf
                    model.advance(prev_ttf)
                    
                prediction = model.get_execution_time()
                error = np.nan if outlier else (prediction - exec_time)
                result_rows.append({
                    "real": exec_time,
                    "prediction": prediction,
                    "error": error,
                    "sqr_error": np.square(error),
                    "model": model_name,
                    "duration": prev_duration,
                    "neuro": neuro,
                    # "kernel": kernel_name,
                    # "ttf_bins": ttf_bins,
                    # "window_size": window_size,
                })

results = pd.DataFrame(result_rows)
for col in ("model", ):  # "kernel"):
    results[col] = results[col].astype("category")

results.to_parquet("./full_validation_ignore_outliers.parquet")
results


# remove outliers from curves
# include old edgedroid
# choose the best models and carry this forward - experimentation, evaluation

Run:   0%|          | 0/40 [00:00<?, ?it/s]

2024-02-11 20:29:41.800 | DEBUG    | edgedroid.models.timings.curve:__init__:114 - Curve fitting aggregation function: outliers_to_nan_then_mean
2024-02-11 20:29:41.838 | INFO     | edgedroid.models.timings.curve:__init__:139 - Fitting execution time functions to data...
2024-02-11 20:29:41.889 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [-inf, 5.0): a * x^b + c (MSE: 0.04, prev. MSE 0.04)
2024-02-11 20:29:41.922 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [5.0, 9.0): a * x^b + c (MSE: 0.09, prev. MSE 0.09)
2024-02-11 20:29:41.925 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [9.0, inf): a * x^b + c (MSE: 0.05, prev. MSE 0.05)
2024-02-11 20:29:41.932 | DEBUG    | edgedroid.models.timings.curve:__init__:114 - Curve fitting aggregation function: outliers_to_nan_then_mean
2024-02-11 20:29:42.069 | INFO     | edgedroid.models.timings.curve:__in

,real,prediction,error,sqr_error,model,duration,neuro
0,3.029,4.368085,1.339085,1.793150,1st order,"[-inf, 5.0)","[-inf, 0.5)"
1,3.029,5.261648,2.232648,4.984719,1st order (median),"[-inf, 5.0)","[-inf, 0.5)"
2,3.029,5.873496,2.844496,8.091159,1st order (mean),"[-inf, 5.0)","[-inf, 0.5)"
3,3.029,5.253714,2.224714,4.949354,a * x^b + c,"[-inf, 5.0)","[-inf, 0.5)"
4,3.029,5.056649,2.027649,4.111359,a * x^2 + b * x + c,"[-inf, 5.0)","[-inf, 0.5)"
...,...,...,...,...,...,...,...
60835,4.241,3.514335,-0.726665,0.528042,a * x^2 + b * x + c,"[9.0, inf)","[0.5, inf)"
60836,4.241,3.173563,-1.067437,1.139422,a * x^3 + b * x^2 + c * x + d,"[9.0, inf)","[0.5, inf)"
60837,4.241,4.017809,-0.223191,0.049814,a * e^x + b,"[9.0, inf)","[0.5, inf)"
60838,4.241,3.246007,-0.994993,0.990010,multi curve,"[9.0, inf)","[0.5, inf)"


In [4]:
results.groupby(["model", "neuro", "duration"], observed=True)["sqr_error"].mean().sort_values(ascending=True)

model                          neuro        duration   
a * x^2 + b * x + c            [-inf, 0.5)  [9.0, inf)      2.823796
a * x^b + c                    [-inf, 0.5)  [9.0, inf)      2.824475
a * x^3 + b * x^2 + c * x + d  [-inf, 0.5)  [9.0, inf)      2.828998
multi curve                    [-inf, 0.5)  [9.0, inf)      2.868345
a * e^x + b                    [-inf, 0.5)  [9.0, inf)      2.936773
legacy                         [-inf, 0.5)  [9.0, inf)      3.272545
multi curve                    [-inf, 0.5)  [5.0, 9.0)      3.371066
a * x^3 + b * x^2 + c * x + d  [-inf, 0.5)  [5.0, 9.0)      3.417326
a * e^x + b                    [-inf, 0.5)  [5.0, 9.0)      3.417669
a * x^b + c                    [-inf, 0.5)  [5.0, 9.0)      3.438045
a * x^2 + b * x + c            [-inf, 0.5)  [5.0, 9.0)      3.448191
a * e^x + b                    [0.5, inf)   [5.0, 9.0)      3.504752
a * x^3 + b * x^2 + c * x + d  [0.5, inf)   [5.0, 9.0)      3.505638
a * x^b + c                    [0.5, inf)   [5.

In [5]:
results.groupby(["model"], observed=True)["sqr_error"].describe().sort_values(by="mean", ascending=True)

,count,mean,std,min,25%,50%,75%,max
model,,,,,,,,
a * x^3 + b * x^2 + c * x + d,6083.0,3.664289,5.590054,1.169998e-06,0.425354,1.809720,4.722671,76.188207
multi curve,6083.0,3.667480,5.020740,1.863419e-09,0.417269,1.825393,5.162923,71.584008
a * e^x + b,6083.0,3.687174,5.575490,8.330463e-08,0.405555,1.840460,4.618087,74.993454
a * x^b + c,6083.0,3.689260,5.683410,2.375562e-08,0.430587,1.755153,4.603407,74.739798
a * x^2 + b * x + c,6083.0,3.692314,5.711395,8.166323e-09,0.429894,1.799450,4.588416,76.511234
1st order (median),6083.0,3.969279,6.069358,6.192399e-08,0.456102,2.314209,4.775917,72.082642
1st order (mean),6083.0,4.489805,5.435193,1.201064e-07,0.809556,2.674773,6.238325,61.955413
legacy,6083.0,5.230826,9.249210,0.000000e+00,0.344569,1.718721,6.262506,93.934864
1st order,6083.0,14.362380,64.858188,6.627907e-09,0.882446,3.918447,12.077556,2466.298236
